# Computer Vision 2022 2nd Lab - Part 2: Εντοπισμός Χωρο-χρονικών Σημείων Ενδιαφέροντος και ΕξΧαρακτηριστικών σε Βίντεο Ανθρωπίνων Δράσεων<br>

Ηλιόπουλος Γεώργιος - 03118815 <br>
Σερλής Αναστάσιος Εμανουήλ - 03118125

In [1]:
from cv22_lab2_2_utils import read_video, show_detection, orientation_histogram, bag_of_words, svm_train_test
from os import listdir
from os.path import isfile, join
import numpy as np
import scipy
from scipy import signal
from scipy import ndimage as nd
import cv2
from scipy.ndimage import convolve1d

In [2]:
#function to make beep so we know the cell finished running
from IPython.display import Audio
sound = listdir('./audio-numbers/')


def notice(n):
    sound_file = './audio-numbers/' + str(n) + '.wav'
    return Audio(sound_file, autoplay = True)

In [3]:
#reading files name
walk_videos = listdir('./walking/')
run_videos = listdir('./running/')
box_videos = listdir('./boxing/')

#conneting file name to path
for i in range (16):
    walk_videos[i] = './walking/' + walk_videos[i]
    run_videos[i] = './running/' + run_videos[i]
    box_videos[i] = './boxing/' + box_videos[i]

print("There are", str(len(walk_videos)), "videos for walking,", str(len(run_videos)), \
      "videos for running and", str(len(box_videos)), "videos for boxing.")

There are 16 videos for walking, 16 videos for running and 16 videos for boxing.


In [4]:
#importing 1 video from each catecory
walk_video1 = read_video(walk_videos[1], 200)
run_video1 = read_video(run_videos[1], 200)
box_video1 = read_video(box_videos[1], 200)

### 2.1: Χωρο-χρονικά Σημεία Ενδιαφέροντος

In [5]:
In = walk_video1
sigma = 4
tau = 1.5
s = 2
k = 0.005
N = 600

n = int(2 * np.ceil(3 * sigma) + 1) # kernel size
Gauss_1D = cv2.getGaussianKernel(n, sigma)
Gauss_1D = Gauss_1D.reshape(Gauss_1D.shape[0])
In_2 = convolve1d(In, Gauss_1D, axis = 0)

In_3 = convolve1d(In_2, Gauss_1D, axis = 1)

n = int(2 * np.ceil(3 * tau) + 1) # kernel size
Gauss_t = cv2.getGaussianKernel(n, tau)
Gauss_t = Gauss_t.reshape(Gauss_t.shape[0])
In_4 = convolve1d(In_3, Gauss_t, axis = 2)

In [6]:
def space_time_Gaus (In, sigma, tau):
    
    n = int(2 * np.ceil(3 * sigma) + 1) # kernel size
    Gauss_1D = cv2.getGaussianKernel(n, sigma)
    Gauss_2D = Gauss_1D @ Gauss_1D.transpose()
    
    # making Gaussian kernel 3D so we can add another Gaussian Kernel in 3rd axis
    Gauss_2D = Gauss_2D.reshape(Gauss_2D.shape[0], Gauss_2D.shape[1], 1)
    
    # create a temporal Gaussian kernel
    n = int(2 * np.ceil(3 * tau) + 1) # kernel size
    Gauss_time = cv2.getGaussianKernel(n, tau)
    # making Gaussian kernel 3D so we can add the above Gaussian Kernels in 1st and 2nd axis
    Gauss_time = Gauss_time.reshape(1, 1, Gauss_time.shape[0])
    
    # connecting the 3 kernels
    Gauss = nd.convolve(Gauss_time, Gauss_2D)
    L = nd.convolve(In, Gauss)
    
    return L

def space_Gaus (In, sigma):
    
    n = int(2 * np.ceil(3 * sigma) + 1) # kernel size
    Gauss_1D = cv2.getGaussianKernel(n, sigma)
    Gauss_2D = Gauss_1D @ Gauss_1D.transpose()
    
    # making Gaussian kernel 3D so we convolve with video only in space (x, y)
    Gauss_2D = Gauss_2D.reshape(Gauss_2D.shape[0], Gauss_2D.shape[1], 1)
    
    L = nd.convolve(In, Gauss_2D)
    
    return L

In [7]:
def Harris_Detector (In, sigma, tau, s, k):
    
    In = In.astype(np.float32)
    
    L = space_time_Gaus(In, sigma, tau)
    
    # using [-1 0 1] table to calculate derivative in one axis
    dx = np.array([[[-1],[0],[1]]])
    Lx = nd.convolve(L, dx)
    # changing [-1 0 1] table orientation
    dy = np.array([[[-1]],[[0]],[[1]]])
    Ly = nd.convolve(L, dy)
    # changing [-1 0 1] table orientation
    dt = np.array([[[-1,  0,  1]]])
    Lt = nd.convolve(L, dt)
    
    L = np.array([[Lx*Lx, Lx*Ly, Lx*Lt], [Lx*Ly, Ly*Ly, Ly*Lt], [Lx*Lt, Ly*Lt, Lt*Lt]])
    
    #initializing M table
    M = L
    
    for i in range(3):
        for j in range(3):
            M[i,j] = space_time_Gaus(L[i,j], sigma*s, tau*s)
    
    trace = np.trace(M)
    det = M[0,0]*(M[1,1]*M[2,2] - M[1,2]*M[2,1]) - M[0,1]*(M[1,0]*M[2,2] - M[1,2]*M[2,0]) \
        + M[0,2]*(M[1,0]*M[2,1] - M[1,1]*M[2,0])
    H = det - k * trace**3
    
    return H

In [8]:
def Gabor_Detector (In, sigma, tau):
    
    In = In.astype(np.float32)
    
    L = space_Gaus(In, sigma)
    
    w = 4 / tau
    t = np.arange(-2*tau, 2*tau + 1)
    
    h_ev = np.cos(2 * np.pi * t * w) * np.exp(-t**2 / (2 * tau**2))
    h_od = np.sin(2 * np.pi * t * w) * np.exp(-t**2 / (2 * tau**2))
    
    #nrmalazation with L1 norm
    h_ev = h_ev/np.linalg.norm(h_ev, ord=1)
    h_od = h_od/np.linalg.norm(h_od, ord=1)
    
    H = convolve1d(L, h_ev) ** 2 + convolve1d(L, h_od) ** 2
    
    return H

In [9]:
def find_points(H, N, sigma):
    
    max_points = np.dstack(np.unravel_index(np.argsort(H.ravel()), (H.shape[0], H.shape[1], H.shape[2])))
    max_points = max_points.reshape(max_points.shape[1], max_points.shape[2])
    max_points = max_points[-N:]
    
    toReturn = np.array([np.append(pair[:2][::-1],[pair[2], sigma]) for pair in max_points])
        
    return toReturn

In [10]:
def Harris_points (In, sigma, tau, s, k, N):
    H = Harris_Detector (In, sigma, tau, s, k)
    return find_points(H, N, sigma)

def Gabor_points (In, sigma, tau, N):
    H = Gabor_Detector (In, sigma, tau)
    return find_points(H, N, sigma)

In [ ]:
In = walk_video1
sigma = 4
tau = 1.5
s = 2
k = 0.005
N = 600
H_Harris_walk = Harris_Detector (In, sigma, tau, s, k)
H_Gabor_walk = Gabor_Detector (In, sigma, tau)
In = box_video1
H_Harris_box = Harris_Detector (In, sigma, tau, s, k)
H_Gabor_box = Gabor_Detector (In, sigma, tau)
In = run_video1
H_Harris_run = Harris_Detector (In, sigma, tau, s, k)
H_Gabor_run = Gabor_Detector (In, sigma, tau)

#exporting data to plot matrixes H
save('H_Harris_walk.npy', H_Harris_walk)
save('H_Gabor_walk.npy', H_Gabor_walk)
save('H_Harris_box.npy', H_Harris_box)
save('H_Gabor_box.npy', H_Gabor_box)
save('H_Harris_run.npy', H_Harris_run)
save('H_Gabor_run.npy', H_Gabor_run)

In [11]:
In = walk_video1
#show_detection(In, Harris_points (In, sigma, tau, s, k, N), save_path=r"./frames/Harris_walk_frames")
In = run_video1
#show_detection(In, Harris_points (In, sigma, tau, s, k, N), save_path=r"./frames/Harris_run_frames")
In = box_video1
#show_detection(In, Harris_points (In, sigma, tau, s, k, N), save_path=r"./frames/Harris_box_frames")
In = walk_video1
#show_detection(In, Gabor_points (In, sigma, tau, N), save_path=r"./frames/Gabor_walk_frames")
In = run_video1
#show_detection(In, Gabor_points (In, sigma, tau, N), save_path=r"./frames/Gabor_run_frames")
In = box_video1
#show_detection(In, Gabor_points (In, sigma, tau, N), save_path=r"./frames/Gabor_box_frames")

### 2.2: Χωρο-χρονικοί Ιστογραφικοί Περιγραφητές

In [12]:
#2.2.1
#dy, dx, _ = np.gradient(In)

def compute_TVL1(prev, cur):
    oflow = cv2.DualTVL1OpticalFlow_create(nscales=1)
    flow = oflow.calc(prev, cur, None)
    return flow

In [13]:
def optical_flow_TVL1(In):
    
    In = In.astype(np.uint8)
    
    #initializing np arrays for flow in x and y
    x_f = np.empty((In.shape[0], In.shape[1], In.shape[2]))
    y_f = np.empty((In.shape[0], In.shape[1], In.shape[2]))
    
    #converting in uint8 type 
    
    for t in range(In.shape[2]):
        
        #if border
        if t == In.shape[2] - 1:
            t = t - 1
        
        prev = In[:,:,t]
        cur = In[:,:,t+1]
        
        H = compute_TVL1(prev, cur)
        
        x_f[:, :, t] = H[:, :, 1]
        y_f[:, :, t] = H[:, :, 0]
        
    return (x_f, y_f)

In [14]:
#2.2.2
def HOG (In, interest_points, nbins=9, n=3, m=3):
    
    dy, dx, _ = np.gradient(In)
    
    max_y = In.shape[0]
    max_x = In.shape[1]
    Y = []
    
    for H in interest_points:
        
        #seperating points' coordinates
        x = H[0]
        y = H[1]
        t = H[2]
        sigma = H[3]
        
        #region 4*sigma around each interest point
        side = int(np.round(4 * sigma))
        
        #using ifs to avoid getting out of image dimentions
        if x - side <= 0:
            x_left = 0
        else:
            x_left = x - side
            
        if x + side + 1 >= max_x:
            x_right = max_x
        else:
            x_right = x + side + 1
        
        if y - side <= 0:
            y_up = 0
        else:
            y_up = y - side
            
        if y + side + 1 >= max_y:
            y_down = max_y
        else:
            y_down = y + side + 1
    
        
        Gx = dx[y_up:y_down, x_left:x_right, t]
        Gy = dy[y_up:y_down, x_left:x_right, t]
        
        Y.append(orientation_histogram(Gx, Gy, nbins, np.array([n,m])))
    
    Y = np.array(Y)
    return Y

In [15]:
def HOF (In, interest_points, nbins=9, n=3, m=3):
    
    dx, dy = optical_flow_TVL1(In)
    
    max_y = In.shape[0]
    max_x = In.shape[1]
    Y = []
    
    for H in interest_points:
        
        #seperating points' coordinates
        x = H[0]
        y = H[1]
        t = H[2]
        sigma = H[3]
        
        #region 4*sigma around each interest point
        side = int(np.round(4 * sigma))
        
        #using ifs to avoid getting out of image dimentions
        if x - side <= 0:
            x_left = 0
        else:
            x_left = x - side
            
        if x + side + 1 >= max_x:
            x_right = max_x
        else:
            x_right = x + side + 1
        
        if y - side <= 0:
            y_up = 0
        else:
            y_up = y - side
            
        if y + side + 1 >= max_y:
            y_down = max_y
        else:
            y_down = y + side + 1
        
        Gx = dx[y_up:y_down, x_left:x_right, t]
        Gy = dy[y_up:y_down, x_left:x_right, t]
        
        Y.append(orientation_histogram(Gx, Gy, nbins, np.array([n,m])))
    
    Y = np.array(Y)
    return Y

In [16]:
def HOF_HOG (In, interest_points, nbins=9, n=3, m=3):
    hof = HOF(In, interest_points, nbins=9, n=3, m=3)
    hog = HOG(In, interest_points, nbins=9, n=3, m=3)
    return np.concatenate((hog, hof))

In [17]:
In = walk_video1
sigma = 4
tau = 1.5
s = 2
k = 0.005
N = 600
points = Gabor_points (In, sigma, tau, N)

### 2.2: Κατασκευή Bag of Visual Words και χρήση Support Vector Machines για την ταξινόμηση δράσεων

In [18]:
#2.3.1

#seperating videos into test and train sets as the traininng_videos.txt describes
training_videos_txt = open('traininng_videos.txt', 'r')
training_videos_names = training_videos_txt.readlines()

training_videos = []
test_videos = []
training_labels= []
test_labels= []
test_videos_name = []

for name in listdir('./walking/'):
    #print('./walking/' + name)
    if name + '\n' in training_videos_names:
        vid = read_video('./walking/' + name , 200)
        training_videos.append(vid)
        training_labels.append('walk')
    else:
        vid = read_video('./walking/' + name , 200)
        test_videos.append(vid)
        test_labels.append('walk')
        test_videos_name.append(name)

        
for name in listdir('./boxing/'):
    #print('./walking/' + name)
    if name + '\n' in training_videos_names:
        vid = read_video('./boxing/' + name , 200)
        training_videos.append(vid)
        training_labels.append('box')
    else:
        vid = read_video('./boxing/' + name , 200)
        test_videos.append(vid)
        test_labels.append('box')
        test_videos_name.append(name)


for name in listdir('./running/'):
    #print('./walking/' + name)
    if name + '\n' in training_videos_names:
        vid = read_video('./running/' + name , 200)
        training_videos.append(vid)
        training_labels.append('run')
    else:
        vid = read_video('./running/' + name , 200)
        test_videos.append(vid)
        test_labels.append('run')
        test_videos_name.append(name)

In [19]:
#2.3.2 and 2.3.3
#Harris/HOG
interest_Harris_points_training = []
desc_HOG_Harris_train = []
sigma = 4
tau = 1.5
s = 2
k = 0.005
N = 600
n = int(np.round(4*sigma))
m = int(np.round(4*sigma))
for Input in training_videos:
    interest_points = Harris_points(Input, sigma, tau, s, k, N)
    interest_Harris_points_training.append(interest_points)
    desc = HOG(In, interest_points, 9, n, m)
    desc_HOG_Harris_train.append(desc)

In [20]:
interest_Harris_points_test = []
desc_HOG_Harris_test = []
for Input in test_videos:
    interest_points = Harris_points(Input, sigma, tau, s, k, N)
    interest_Harris_points_test.append(interest_points)
    desc = HOG(In, interest_points, 9, n, m)
    desc_HOG_Harris_test.append(desc)

In [21]:
#Bag of Words for Harris/HOG
bow_train_HOG_Harris, bow_test_HOG_Harris = bag_of_words(desc_HOG_Harris_train, desc_HOG_Harris_test, 400)
notice(1)

In [22]:
#Harris/HOF
desc_HOF_Harris_train = []
i = 0
for Input in training_videos:
    desc = HOF(In, interest_Harris_points_training[i], 9, n, m)
    desc_HOF_Harris_train.append(desc)
    i += 1

i = 0
desc_HOF_Harris_test = []
for Input in test_videos:
    desc = HOG(In, interest_Harris_points_test[i], 9, n, m)
    desc_HOF_Harris_test.append(desc)
    i += 1

In [23]:
#Bag of Words for Harris/HOF
bow_train_HOF_Harris, bow_test_HOF_Harris = bag_of_words(desc_HOF_Harris_train, desc_HOF_Harris_test, 400)
notice(2)

In [24]:
#Harris/HOF_HOG
desc_HOF_HOG_Harris_train = []
i = 0
for Input in training_videos:
    desc = HOF_HOG(In, interest_Harris_points_training[i], 9, n, m)
    desc_HOF_HOG_Harris_train.append(desc)
    i += 1

i = 0
desc_HOF_HOG_Harris_test = []
for Input in test_videos:
    desc = HOF_HOG(In, interest_Harris_points_test[i], 9, n, m)
    desc_HOF_HOG_Harris_test.append(desc)
    i += 1

In [25]:
#Bag of Words for Harris/HOF_HOG
bow_train_HOF_HOG_Harris, bow_test_HOF_HOG_Harris = bag_of_words(desc_HOF_HOG_Harris_train, desc_HOF_HOG_Harris_test, 400)
notice(3)

In [26]:
#Gabor/HOG
interest_Gabor_points_training = []
desc_HOG_Gabor_train = []
for Input in training_videos:
    interest_points = Gabor_points(Input, sigma, tau, N)
    interest_Gabor_points_training.append(interest_points)
    desc = HOG(In, interest_points, 9, n, m)
    desc_HOG_Gabor_train.append(desc)

In [27]:
interest_Gabor_points_test = []
desc_HOG_Gabor_test = []
for Input in test_videos:
    interest_points = Gabor_points(Input, sigma, tau, N)
    interest_Gabor_points_test.append(interest_points)
    desc = HOG(In, interest_points, 9, n, m)
    desc_HOG_Gabor_test.append(desc)

In [28]:
#Bag of Words for Gabor/HOG
bow_train_HOG_Gabor, bow_test_HOG_Gabor = bag_of_words(desc_HOG_Gabor_train, desc_HOG_Gabor_test, 400)
notice(4)

In [29]:
#Gabor/HOF
desc_HOF_Gabor_train = []
i = 0
for Input in training_videos:
    desc = HOF(In, interest_Gabor_points_training[i], 9, n, m)
    desc_HOF_Gabor_train.append(desc)
    i += 1

i = 0
desc_HOF_Gabor_test = []
for Input in test_videos:
    desc = HOG(In, interest_Gabor_points_test[i], 9, n, m)
    desc_HOF_Gabor_test.append(desc)
    i += 1

In [40]:
#Bag of Words for Gabor/HOF
bow_train_HOF_Gabor, bow_test_HOF_Gabor = bag_of_words(desc_HOF_Gabor_train, desc_HOF_Gabor_test, 20)
notice(5)

In [31]:
#Gabor/HOF_HOG
desc_HOF_HOG_Gabor_train = []
i = 0
for Input in training_videos:
    desc = HOF_HOG(In, interest_Gabor_points_training[i], 9, n, m)
    desc_HOF_HOG_Gabor_train.append(desc)
    i += 1

i = 0
desc_HOF_HOG_Gabor_test = []
for Input in test_videos:
    desc = HOF_HOG(In, interest_Gabor_points_test[i], 9, n, m)
    desc_HOF_HOG_Gabor_test.append(desc)
    i += 1

In [33]:
#Bag of Words for Gabor/HOF_HOG
bow_train_HOF_HOG_Gabor, bow_test_HOF_HOG_Gabor = bag_of_words(desc_HOF_HOG_Gabor_train, desc_HOF_HOG_Gabor_test, 400)
notice(6)

In [34]:
combo = 'detector:\tHarris \ndescriptor:\tHOG'
bow_train = bow_train_HOG_Harris
bow_test = bow_test_HOG_Harris
accuracy, pred = svm_train_test(bow_train, training_labels, bow_test, test_labels)
print('\033[1m' + combo + '\033[0m')
print()
print('The accuracy for the combination is ' + str(format(accuracy*100, ".3f")) + "%")
print()
print("Here is a table with detailed results:")
print()
k = test_videos_name
a = test_labels
b = pred
c = a == b
print("Video Name \t \t \t Correct Prediction")

for i in range(len(a)):
    print(k[i] + '\t ' + a[i] + '\t ' + b[i]+ '\t ' + str(c[i]))

detector:	Harris 
descriptor:	HOG

The accuracy for the combination is 41.667%

Here is a table with detailed results:

Video Name 	 	 	 Correct Prediction
person07_walking_d4_uncomp.avi	 walk	 run	 False
person08_walking_d2_uncomp.avi	 walk	 walk	 True
person11_walking_d1_uncomp.avi	 walk	 run	 False
person20_walking_d3_uncomp.avi	 walk	 run	 False
person06_boxing_d1_uncomp.avi	 box	 run	 False
person07_boxing_d3_uncomp.avi	 box	 run	 False
person11_boxing_d2_uncomp.avi	 box	 box	 True
person15_boxing_d4_uncomp.avi	 box	 run	 False
person10_running_d2_uncomp.avi	 run	 box	 False
person20_running_d1_uncomp.avi	 run	 run	 True
person24_running_d3_uncomp.avi	 run	 run	 True
person25_running_d4_uncomp.avi	 run	 run	 True


In [35]:
combo = 'detector:\tHarris \ndescriptor:\tHOF'
bow_train = bow_train_HOF_Harris
bow_test = bow_test_HOF_Harris
accuracy, pred = svm_train_test(bow_train, training_labels, bow_test, test_labels)
print('\033[1m' + combo + '\033[0m')
print()
print('The accuracy for the combination is ' + str(format(accuracy*100, ".3f")) + "%")
print()
print("Here is a table with detailed results:")
print()
k = test_videos_name
a = test_labels
b = pred
c = a == b
print("Video Name \t \t \t Correct Prediction")

for i in range(len(a)):
    print(k[i] + '\t ' + a[i] + '\t ' + b[i]+ '\t ' + str(c[i]))

detector:	Harris 
descriptor:	HOF

The accuracy for the combination is 33.333%

Here is a table with detailed results:

Video Name 	 	 	 Correct Prediction
person07_walking_d4_uncomp.avi	 walk	 walk	 True
person08_walking_d2_uncomp.avi	 walk	 walk	 True
person11_walking_d1_uncomp.avi	 walk	 walk	 True
person20_walking_d3_uncomp.avi	 walk	 walk	 True
person06_boxing_d1_uncomp.avi	 box	 walk	 False
person07_boxing_d3_uncomp.avi	 box	 walk	 False
person11_boxing_d2_uncomp.avi	 box	 walk	 False
person15_boxing_d4_uncomp.avi	 box	 walk	 False
person10_running_d2_uncomp.avi	 run	 walk	 False
person20_running_d1_uncomp.avi	 run	 walk	 False
person24_running_d3_uncomp.avi	 run	 walk	 False
person25_running_d4_uncomp.avi	 run	 walk	 False


In [36]:
combo = 'detector:\tHarris \ndescriptor:\tHOF and HOG'
bow_train = bow_train_HOF_HOG_Harris
bow_test = bow_test_HOF_HOG_Harris
accuracy, pred = svm_train_test(bow_train, training_labels, bow_test, test_labels)
print('\033[1m' + combo + '\033[0m')
print()
print('The accuracy for the combination is ' + str(format(accuracy*100, ".3f")) + "%")
print()
print("Here is a table with detailed results:")
print()
k = test_videos_name
a = test_labels
b = pred
c = a == b
print("Video Name \t \t \t Correct Prediction")

for i in range(len(a)):
    print(k[i] + '\t ' + a[i] + '\t ' + b[i]+ '\t ' + str(c[i]))

detector:	Harris 
descriptor:	HOF and HOG

The accuracy for the combination is 83.333%

Here is a table with detailed results:

Video Name 	 	 	 Correct Prediction
person07_walking_d4_uncomp.avi	 walk	 walk	 True
person08_walking_d2_uncomp.avi	 walk	 walk	 True
person11_walking_d1_uncomp.avi	 walk	 walk	 True
person20_walking_d3_uncomp.avi	 walk	 run	 False
person06_boxing_d1_uncomp.avi	 box	 box	 True
person07_boxing_d3_uncomp.avi	 box	 walk	 False
person11_boxing_d2_uncomp.avi	 box	 box	 True
person15_boxing_d4_uncomp.avi	 box	 box	 True
person10_running_d2_uncomp.avi	 run	 run	 True
person20_running_d1_uncomp.avi	 run	 run	 True
person24_running_d3_uncomp.avi	 run	 run	 True
person25_running_d4_uncomp.avi	 run	 run	 True


In [37]:
combo = 'detector:\tGabor \ndescriptor:\tHOG'
bow_train = bow_train_HOG_Gabor
bow_test = bow_test_HOG_Gabor
accuracy, pred = svm_train_test(bow_train, training_labels, bow_test, test_labels)
print('\033[1m' + combo + '\033[0m')
print()
print('The accuracy for the combination is ' + str(format(accuracy*100, ".3f")) + "%")
print()
print("Here is a table with detailed results:")
print()
k = test_videos_name
a = test_labels
b = pred
c = a == b
print("Video Name \t \t \t Correct Prediction")

for i in range(len(a)):
    print(k[i] + '\t ' + a[i] + '\t ' + b[i]+ '\t ' + str(c[i]))

detector:	Gabor 
descriptor:	HOG

The accuracy for the combination is 41.667%

Here is a table with detailed results:

Video Name 	 	 	 Correct Prediction
person07_walking_d4_uncomp.avi	 walk	 run	 False
person08_walking_d2_uncomp.avi	 walk	 run	 False
person11_walking_d1_uncomp.avi	 walk	 run	 False
person20_walking_d3_uncomp.avi	 walk	 run	 False
person06_boxing_d1_uncomp.avi	 box	 run	 False
person07_boxing_d3_uncomp.avi	 box	 box	 True
person11_boxing_d2_uncomp.avi	 box	 run	 False
person15_boxing_d4_uncomp.avi	 box	 run	 False
person10_running_d2_uncomp.avi	 run	 run	 True
person20_running_d1_uncomp.avi	 run	 run	 True
person24_running_d3_uncomp.avi	 run	 run	 True
person25_running_d4_uncomp.avi	 run	 run	 True


In [41]:
combo = 'detector:\tGabor \ndescriptor:\tHOF'
bow_train = bow_train_HOF_Gabor
bow_test = bow_test_HOF_Gabor
accuracy, pred = svm_train_test(bow_train, training_labels, bow_test, test_labels)
print('\033[1m' + combo + '\033[0m')
print()
print('The accuracy for the combination is ' + str(format(accuracy*100, ".3f")) + "%")
print()
print("Here is a table with detailed results:")
print()
k = test_videos_name
a = test_labels
b = pred
c = a == b
print("Video Name \t \t \t Correct Prediction")

for i in range(len(a)):
    print(k[i] + '\t ' + a[i] + '\t ' + b[i]+ '\t ' + str(c[i]))

detector:	Gabor 
descriptor:	HOF

The accuracy for the combination is 33.333%

Here is a table with detailed results:

Video Name 	 	 	 Correct Prediction
person07_walking_d4_uncomp.avi	 walk	 walk	 True
person08_walking_d2_uncomp.avi	 walk	 walk	 True
person11_walking_d1_uncomp.avi	 walk	 walk	 True
person20_walking_d3_uncomp.avi	 walk	 walk	 True
person06_boxing_d1_uncomp.avi	 box	 walk	 False
person07_boxing_d3_uncomp.avi	 box	 walk	 False
person11_boxing_d2_uncomp.avi	 box	 walk	 False
person15_boxing_d4_uncomp.avi	 box	 walk	 False
person10_running_d2_uncomp.avi	 run	 walk	 False
person20_running_d1_uncomp.avi	 run	 walk	 False
person24_running_d3_uncomp.avi	 run	 walk	 False
person25_running_d4_uncomp.avi	 run	 walk	 False


In [39]:
combo = 'detector:\tGabor \ndescriptor:\tHOF and HOG'
bow_train = bow_train_HOF_HOG_Gabor
bow_test = bow_test_HOF_HOG_Gabor
accuracy, pred = svm_train_test(bow_train, training_labels, bow_test, test_labels)
print('\033[1m' + combo + '\033[0m')
print()
print('The accuracy for the combination is ' + str(format(accuracy*100, ".3f")) + "%")
print()
print("Here is a table with detailed results:")
print()
k = test_videos_name
a = test_labels
b = pred
c = a == b
print("Video Name \t \t \t Correct Prediction")

for i in range(len(a)):
    print(k[i] + '\t ' + a[i] + '\t ' + b[i]+ '\t ' + str(c[i]))

detector:	Gabor 
descriptor:	HOF and HOG

The accuracy for the combination is 91.667%

Here is a table with detailed results:

Video Name 	 	 	 Correct Prediction
person07_walking_d4_uncomp.avi	 walk	 walk	 True
person08_walking_d2_uncomp.avi	 walk	 walk	 True
person11_walking_d1_uncomp.avi	 walk	 run	 False
person20_walking_d3_uncomp.avi	 walk	 walk	 True
person06_boxing_d1_uncomp.avi	 box	 box	 True
person07_boxing_d3_uncomp.avi	 box	 box	 True
person11_boxing_d2_uncomp.avi	 box	 box	 True
person15_boxing_d4_uncomp.avi	 box	 box	 True
person10_running_d2_uncomp.avi	 run	 run	 True
person20_running_d1_uncomp.avi	 run	 run	 True
person24_running_d3_uncomp.avi	 run	 run	 True
person25_running_d4_uncomp.avi	 run	 run	 True
